In [4]:
class deal():
    '''Класс, хранящий информацию о текущих и проведенных сделок'''
    def __init__(self):
        self.is_open = False
        
        self.opentime = 0
        self.closetime = 0
        
        self.openprice = 0
        self.closeprice = 0
        self.futures = 0
        
        self.cost = 0
        self.profit = 0
        
        self.dealyield = 0
        
        self.is_profitable = False
    def __str__(self):
        return f'openprice: {self.openprice}\ncloseprice: {self.closeprice}\nfutures: {self.futures}\ncost: {self.cost}\nprofit: {self.profit}\nyield of the deal: {self.dealyield} %\n'
    def __repr__(self):
        return f'openprice: {self.openprice}\ncloseprice: {self.closeprice}\nfutures: {self.futures}\ncost: {self.cost}\nprofit: {self.profit}\nyield of the deal: {self.dealyield} %\n' 
    
    def open_the_deal(self,openprice,share,budget,opentime):
        self.is_open = True
        self.opentime = opentime
        self.openprice = openprice
        self.futures = (share*budget)//self.openprice
        self.cost = self.futures*self.openprice
    
    def close_the_deal(self,closeprice,closetime):
        self.closetime = closetime
        self.closeprice = closeprice
        self.profit = self.futures*closeprice
        self.dealyield = round(((self.profit-self.cost)/self.cost)*100,3)
        if self.dealyield > 0:
            self.is_profitable = True
    def get_openprice(self): return self.openprice
    def get_cost(self): return self.cost
    def get_profit(self): return self.profit

In [5]:
class bot():
    '''Класс симулирующий процесс принятия решения о покупке и продаже контрактов в соответствии с поступающими данными'''
    def __init__(self,budget,ema_thr,share,thr):
        self.budget = budget
        self.exit_thr = 0
        self.thr = thr
        self.ema_thr = ema_thr
        self.share = share
        self.current_deal = deal()
        self.deallist = []
    def __str__(self):
        return f'budget: {self.budget} thr: {self.thr}'
    def __repr__(self):
        return f'budget: {budget}'
    def ticker(self,present,predict,ema,time):
        if self.current_deal.is_open:
            return self.exit(present,predict,ema,time)
        else:
            return self.entry(present,predict,ema,time)
            
    def entry(self,present,predict,ema,time):
        '''Функция выхода на рынок'''
        if (predict > present) and (present-ema > self.ema_thr):
            self.current_deal.open_the_deal(present,self.share,self.budget,time)
            self.exit_thr = present
            self.budget -= self.current_deal.get_cost()
            return present
        else:
            return None
    def exit(self,present,predict,ema,time):
        '''Функция закрытия сделки'''
        if (self.exit_thr - present > self.thr) or (present  < ema):
            self.current_deal.close_the_deal(present,time)
            self.budget += self.current_deal.get_profit()
            self.deallist.append(self.current_deal)
            self.current_deal = deal()
        elif present < self.exit_thr:
            pass
        else:
            self.exit_thr = self.exit_thr + (present - self.exit_thr)/2
        return present
    def get_stat(self):
        '''Функция для получения общей статистики о проведенных сделках'''
        deals_amount = len(self.deallist)
        profitable = len([i for i in self.deallist if i.is_profitable])
        unprofitable = len([i for i in self.deallist if not i.is_profitable])
        meanyield = round(sum([i.dealyield for i in self.deallist if i.is_profitable])/profitable,3)
        meanloss = round(sum([i.dealyield for i in self.deallist if not i.is_profitable])/profitable,3)
        
        return f'deals amount: {deals_amount}\nprofitable: {profitable}\nunprofitable: {unprofitable}\nmean yield: {meanyield}\nmean loss: {meanloss}'
    def get_dataframe(self):
        '''Функция для выгрузки данных о проведенных сделках в виде таблицы'''
        deallist = [(i.opentime,i.closetime,i.openprice,i.closeprice,i.cost,i.profit,i.dealyield,i.is_profitable) for i in self.deallist]
        df = pd.DataFrame(deallist,columns=['opentime','closetime','openprice','closeprice','cost','profit','dealyield','is_profitable'])
        return df
    